In [1]:
from google.colab import drive
from sklearn.model_selection import train_test_split
from keras import models
import pandas as pd
from keras import layers
import cv2
import numpy as np
from keras.applications.vgg16 import VGG16
from keras import optimizers
from keras.utils import to_categorical
import os

Using TensorFlow backend.


In [2]:
drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


In [0]:
!unzip "/content/drive/My Drive/trashnet_newaug.zip" -d "/content/drive/My Drive/trash_new"

Archive:  /content/drive/My Drive/trashnet_newaug.zip
   creating: /content/drive/My Drive/trash_new/paper/
  inflating: /content/drive/My Drive/trash_new/paper/augmented_image_0.jpg  
  inflating: /content/drive/My Drive/trash_new/paper/augmented_image_1.jpg  
  inflating: /content/drive/My Drive/trash_new/paper/augmented_image_10.jpg  
  inflating: /content/drive/My Drive/trash_new/paper/augmented_image_100.jpg  
  inflating: /content/drive/My Drive/trash_new/paper/augmented_image_101.jpg  
  inflating: /content/drive/My Drive/trash_new/paper/augmented_image_102.jpg  
  inflating: /content/drive/My Drive/trash_new/paper/augmented_image_103.jpg  
  inflating: /content/drive/My Drive/trash_new/paper/augmented_image_104.jpg  
  inflating: /content/drive/My Drive/trash_new/paper/augmented_image_105.jpg  
  inflating: /content/drive/My Drive/trash_new/paper/augmented_image_106.jpg  
  inflating: /content/drive/My Drive/trash_new/paper/augmented_image_107.jpg  
  inflating: /content/drive/

In [0]:
path= "/content/drive/My Drive/trash_new"

In [4]:
dirs=os.listdir(path)
print(dirs)

['paper', 'plastic', 'cardboard', 'glass', 'metal']


In [0]:
GLASS = 0 #3
PAPER = 1 #2
CARDBOARD = 2 #4
PLASTIC = 3 #1
METAL = 4 #6
TRASH = 5 #5

DIM1 = 384
DIM2 = 512

In [5]:
objects=[]
labels=[]
x={'0':1,'1':3,'2':2,'3':0,'4':4}
i=0
for file in dirs:
  print(file)
 
  c=0
  if(i<6):
    for images in os.listdir("/content/drive/My Drive/trash_new/"+file):
      img1=cv2.imread("/content/drive/My Drive/trash_new/"+file+"/"+images)
      img1=cv2.resize(img1,(150,150))
      if(c%50==0):
        print(c)
      c=c+1
      objects.append(np.asarray(img1))
      labels.append(x[str(i)])
  i=i+1

    

paper
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
plastic
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
cardboard
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
glass
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
metal
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900


In [0]:
from keras.preprocessing.image import ImageDataGenerator
dataset_path = "/content/drive/My Drive/dataset-resized/"
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        #horizontal_flip=True,
        #fill_mode='nearest',
        validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(128, 128),
        color_mode="rgb",
        batch_size=64,
        class_mode="categorical",
        shuffle=True,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(128, 128),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True,#same as in train generator
        subset='validation')

Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [0]:
print((np.stack(labels)).shape)

(4902,)


In [0]:
labels=to_categorical(labels)

In [0]:
print(objects[2055].shape)

(128, 128, 3)


In [7]:
obj=np.stack( objects )
print(obj.shape)

(4902, 150, 150, 3)


In [0]:
print(obj.shape)
print(labels.shape)
np.save('objects_data',obj)

(2527, 384, 512, 3)
(2527, 6)


In [0]:
from google.colab import files
#files.download('objects_data.npy')

In [0]:
labels=np.load('/content/drive/My Drive/dataset-resized/objects_labels.npy')

object1=np.load('/content/drive/My Drive/dataset-resized/objects_data.npy')

FileNotFoundError: ignored

In [0]:
print(labels.shape)

(4902, 5)


In [0]:
np.save('objects_labels',labels)
files.download('objects_labels.npy')

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(obj,labels, test_size=0.3)

In [0]:
from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [44]:

from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,BatchNormalization

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.2))
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  
model.add(Dense(128))
model.add(Activation('relu'))
#model.add(Dropout(0.2))
#model.add(Dense(64))
#model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.adam(lr=1e-4),
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
batch_normalization_11 (Batc (None, 148, 148, 32)      128       
_________________________________________________________________
activation_23 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 146, 146, 64)      18496     
_________________________________________________________________
batch_normalization_12 (Batc (None, 146, 146, 64)      256       
_________________________________________________________________
activation_24 (Activation)   (None, 146, 146, 64)      0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 73, 73, 64)        0         
__________

In [45]:
model.fit(X_train, y_train, epochs=50,validation_data=(X_test,y_test), batch_size=300)

Train on 3431 samples, validate on 1471 samples
Epoch 1/50
3431/3431 [==============================] - 19s 6ms/step - loss: 5.2833 - acc: 0.2955 - val_loss: 4.1473 - val_acc: 0.3916
Epoch 2/50
3431/3431 [==============================] - 10s 3ms/step - loss: 3.3882 - acc: 0.4491 - val_loss: 1.3861 - val_acc: 0.3950
Epoch 3/50
3431/3431 [==============================] - 10s 3ms/step - loss: 1.3508 - acc: 0.4177 - val_loss: 1.2939 - val_acc: 0.4398
Epoch 4/50
3431/3431 [==============================] - 10s 3ms/step - loss: 1.2528 - acc: 0.4509 - val_loss: 1.2291 - val_acc: 0.4847
Epoch 5/50
3431/3431 [==============================] - 10s 3ms/step - loss: 1.1739 - acc: 0.4978 - val_loss: 1.1750 - val_acc: 0.5010
Epoch 6/50
3431/3431 [==============================] - 10s 3ms/step - loss: 1.0659 - acc: 0.5538 - val_loss: 1.1070 - val_acc: 0.5500
Epoch 7/50
3431/3431 [==============================] - 10s 3ms/step - loss: 0.9888 - acc: 0.5940 - val_loss: 1.0657 - val_acc: 0.5806
Epoch 8

In [16]:
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f1550959e48> False
<keras.layers.convolutional.Conv2D object at 0x7f15509590f0> False
<keras.layers.convolutional.Conv2D object at 0x7f15507e7128> False
<keras.layers.pooling.MaxPooling2D object at 0x7f15507f64a8> False
<keras.layers.convolutional.Conv2D object at 0x7f15507f64e0> False
<keras.layers.convolutional.Conv2D object at 0x7f15507a8278> False
<keras.layers.pooling.MaxPooling2D object at 0x7f155073f8d0> False
<keras.layers.convolutional.Conv2D object at 0x7f155073f438> False
<keras.layers.convolutional.Conv2D object at 0x7f155076d588> False
<keras.layers.convolutional.Conv2D object at 0x7f1550703ba8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f1550730320> False
<keras.layers.convolutional.Conv2D object at 0x7f155071df28> False
<keras.layers.convolutional.Conv2D object at 0x7f15506c9f28> False
<keras.layers.convolutional.Conv2D object at 0x7f15506f56a0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f1550690e10>

In [17]:
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.4))
#model.add(layers.Dense(128, activation='relu'))
#model.add(layers.Dropout(0.4))
model.add(layers.Dense(5, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              8389632   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 5125      
Total params: 23,109,445
Trainable params: 15,474,181
Non-trainable params: 7,635,264
_________________________________________________________________


In [0]:

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.adam(lr=1e-4),
              metrics=['acc'])

In [0]:
X_train=np.asarray(train_generator)


KeyboardInterrupt: ignored

In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50,verbose=2
)

In [21]:
model.fit(X_train, y_train, epochs=30,validation_data=(X_test,y_test), batch_size=300)

Train on 3431 samples, validate on 1471 samples
Epoch 1/30
3431/3431 [==============================] - 14s 4ms/step - loss: 4.7004 - acc: 0.4232 - val_loss: 1.3080 - val_acc: 0.4691
Epoch 2/30
3431/3431 [==============================] - 13s 4ms/step - loss: 1.2995 - acc: 0.4792 - val_loss: 1.1231 - val_acc: 0.5493
Epoch 3/30
3431/3431 [==============================] - 13s 4ms/step - loss: 1.0433 - acc: 0.5911 - val_loss: 0.8824 - val_acc: 0.6499
Epoch 4/30
3431/3431 [==============================] - 13s 4ms/step - loss: 0.7246 - acc: 0.7158 - val_loss: 0.7055 - val_acc: 0.7288
Epoch 5/30
3431/3431 [==============================] - 13s 4ms/step - loss: 0.4951 - acc: 0.8193 - val_loss: 0.5827 - val_acc: 0.7859
Epoch 6/30
3431/3431 [==============================] - 13s 4ms/step - loss: 0.3318 - acc: 0.8764 - val_loss: 0.5153 - val_acc: 0.8314
Epoch 7/30
3431/3431 [==============================] - 13s 4ms/step - loss: 0.2126 - acc: 0.9222 - val_loss: 0.4581 - val_acc: 0.8545
Epoch 8

In [42]:
predictions=model.predict(X_test)
acc = model.evaluate(X_test, y_test)
print(acc[1])
#model.save("/content/drive/My Drive/FER/fer2013/WSP2_final2.h5")

1471/1471 [==============================] - 1s 445us/step
0.8008157710977187
